In [556]:
import pandas as pd
import nltk
import numpy as np
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
import re
import emoji
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split


#1.
df = pd.read_csv (r'/Users/shenxuanyu/Desktop/corona_fake.csv')
#print (df)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shenxuanyu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shenxuanyu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shenxuanyu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/shenxuanyu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shenxuanyu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [456]:
text = df['text'].astype(str)
print(text[0])

You just need to add water, and the drugs and vaccines are ready to be administered. There are two parts to the kit: one holds pellets containing the chemical machinery that synthesises the end product, and the other holds pellets containing instructions that telll the drug which compound to create. Mix two parts together in a chosen combination, add water, and the treatment is ready.


In [457]:
list = []
for e in text:
    e = str(e)
    list.append(e)

1159

In [458]:
tokens = []
for e in text:
    m = nltk.word_tokenize(e)
    tokens.append(m)
type(tokens[0][0])

str

In [459]:
tagged = []
for e in tokens:
    #e = ' '.join(e)
    m = nltk.pos_tag(e)
    tagged.append(m)
    
WNL = WordNetLemmatizer()

In [461]:
def get_wordnet_pos(word): #referrence: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [463]:
lemma_list = []
for e in tokens:
    lemma_ele = []
    for x in e:
        lemma = WNL.lemmatize(x, get_wordnet_pos(x))
        lemma_ele.append(lemma)
    lemma_list.append(lemma_ele)

In [464]:
print(lemma_list[0])

['You', 'just', 'need', 'to', 'add', 'water', ',', 'and', 'the', 'drug', 'and', 'vaccine', 'be', 'ready', 'to', 'be', 'administer', '.', 'There', 'be', 'two', 'part', 'to', 'the', 'kit', ':', 'one', 'hold', 'pellet', 'contain', 'the', 'chemical', 'machinery', 'that', 'synthesis', 'the', 'end', 'product', ',', 'and', 'the', 'other', 'hold', 'pellet', 'contain', 'instruction', 'that', 'telll', 'the', 'drug', 'which', 'compound', 'to', 'create', '.', 'Mix', 'two', 'part', 'together', 'in', 'a', 'chosen', 'combination', ',', 'add', 'water', ',', 'and', 'the', 'treatment', 'be', 'ready', '.']


In [465]:
stopwords = set(stopwords.words('english'))

In [466]:
no_stopwords_list = []
for ele in lemma_list:
    list = []
    for x in ele:
        if x not in stopwords: 
            list.append(x)
    no_stopwords_list.append(list)

In [467]:
print(no_stopwords_list[0])
len(no_stopwords_list)

['You', 'need', 'add', 'water', ',', 'drug', 'vaccine', 'ready', 'administer', '.', 'There', 'two', 'part', 'kit', ':', 'one', 'hold', 'pellet', 'contain', 'chemical', 'machinery', 'synthesis', 'end', 'product', ',', 'hold', 'pellet', 'contain', 'instruction', 'telll', 'drug', 'compound', 'create', '.', 'Mix', 'two', 'part', 'together', 'chosen', 'combination', ',', 'add', 'water', ',', 'treatment', 'ready', '.']


1159

In [528]:
final_list = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
url = 'http'

In [529]:
def text_has_emoji(text):
    for ele in text:
        if ele in emoji.UNICODE_EMOJI:
            return True
        else:
            return False

In [530]:
for ele in no_stopwords_list:#get cleaned text
    list = []
    for x in ele:
        try:
            float(x)
        except ValueError:
            if x.isdigit() == True:
                continue
            elif len(x) <= 2:
                continue
            elif x in punc:
                continue
            elif x in url:
                continue
            elif text_has_emoji(x) == True:
                continue
            else:
                list.append(x)
    final_list.append(list)

In [532]:
final = []#join 
for ele in final_list:
    ele = ' '.join(ele)
    final.append(ele)

In [534]:
df['text_clean'] = final

In [1]:
#print(df.head)

In [536]:
#2.
cvectorizer1 = CountVectorizer(ngram_range=(1, 1), lowercase=True)
cvectorizer2 = CountVectorizer(ngram_range=(1, 2), lowercase=True)
cvectorizer3 = CountVectorizer(ngram_range=(1, 3), lowercase=True)


In [537]:
X = df['text_clean']
X1 = cvectorizer1.fit_transform(X)

In [538]:
X2 = cvectorizer2.fit_transform(X)

In [539]:
X3 = cvectorizer3.fit_transform(X)

In [540]:
tvectorizer1 = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
tvectorizer2 = TfidfVectorizer(ngram_range=(1, 2), lowercase=True)
tvectorizer3 = TfidfVectorizer(ngram_range=(1, 3), lowercase=True)

In [541]:
X4 = tvectorizer1.fit_transform(X)

In [542]:
X5 = tvectorizer2.fit_transform(X)

In [543]:
X6 = tvectorizer3.fit_transform(X)

In [546]:
#3.
logi = LogisticRegressionCV(cv=5, random_state=265, max_iter = 1000, n_jobs = -1)


In [547]:
Y = df['label']

In [548]:
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1, Y, test_size=0.3, random_state=265)
logi.fit(X1_train,Y1_train)
Y1_pred = logi.predict(X1_test)
logi.score(X1_test, Y1_test)

0.9080459770114943

In [549]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y, test_size=0.3, random_state=265)
logi.fit(X2_train,Y2_train)
Y2_pred = logi.predict(X2_test)
logi.score(X2_test, Y2_test)

0.9195402298850575

In [550]:
X3_train, X3_test, Y3_train, Y3_test = train_test_split(X3, Y, test_size=0.3, random_state=265)
logi.fit(X3_train,Y3_train)
Y3_pred = logi.predict(X3_test)
logi.score(X3_test, Y3_test)

0.8994252873563219

In [551]:
X4_train, X4_test, Y4_train, Y4_test = train_test_split(X4, Y, test_size=0.3, random_state=265)
logi.fit(X4_train,Y4_train)
Y4_pred = logi.predict(X4_test)
logi.score(X4_test, Y4_test)

0.9195402298850575

In [552]:
X5_train, X5_test, Y5_train, Y5_test = train_test_split(X5, Y, test_size=0.3, random_state=265)
logi.fit(X5_train,Y5_train)
Y5_pred = logi.predict(X5_test)
logi.score(X5_test, Y5_test)

0.9051724137931034

In [553]:
X6_train, X6_test, Y6_train, Y6_test = train_test_split(X6, Y, test_size=0.3, random_state=265)
logi.fit(X6_train,Y6_train)
Y6_pred = logi.predict(X6_test)
logi.score(X6_test, Y6_test)

0.896551724137931